In [1]:
import numpy as np

In [32]:
# initialize array for storing data
number_data = 338
data = {"id": np.empty([2, number_data], dtype="int32"),
       "thread_id": np.empty([2,number_data], dtype="int32"),
       "sentiment": np.empty([2,number_data], dtype="int32"),
        "whoposts": np.empty([2,number_data], dtype="str"),
       "post_body": np.empty([2,number_data], dtype=object)}

In [33]:
# read annotations from .csv file
import csv
files = ['data/posts_excel_deleted.csv', 'data/posts_excel_1_deleted.csv']
for i in range(len(files)):
    with open(files[i]) as csvfile:
        reader = csv.reader(csvfile)
        count = 0
        for row in reader:
            # exclude the first row, which are column names
            if count != 0:
                #print("Current line:",count)
                data['id'][i, count-1] = count
                data['thread_id'][i, count-1] = row[0]
                data['sentiment'][i, count-1] = row[1]
                data['whoposts'][i, count-1] = row[2]
                data['post_body'][i, count-1] = row[3]
            count += 1

In [141]:
import subprocess
def senti_strength(text):
    cmd = ['java', '-jar', '../senti_strength/SentiStrengthCom.jar', 'sentidata', 
           '../senti_strength/SentStrength_Data/', 'text', text, 
           'sentenceCombineTot','paragraphCombineTot','maxWordsBeforeSentimentToNegate',
           '3',]
    return subprocess.check_output(cmd).decode("utf-8").replace('\n','')

In [122]:
senti_value = np.empty([2,number_data],dtype="int32")
senti_output = np.empty(number_data,dtype="int32")
print(senti_value.shape)
for i in range(data['post_body'].shape[1]):
    output = senti_strength(data['post_body'][0,i])
    values = [int(x) for x in output.split(" ")]
    senti_value[:,i] = values
    if values[0]> -values[1]:
        senti_output[i] = 1
    else:
        senti_output[i] = 0
    if i % 30 == 0:
        print("predicted %d/%d" % (i, number_data))

(2, 338)
predicted 0/338
predicted 30/338
predicted 60/338
predicted 90/338
predicted 120/338
predicted 150/338
predicted 180/338
predicted 210/338
predicted 240/338
predicted 270/338
predicted 300/338
predicted 330/338


In [146]:
print(senti_value.T)
print(senti_output)

[[  0  -7]
 [ 33 -28]
 [  5  -2]
 [  0   0]
 [  3  -6]
 [ 14  -4]
 [  2  -2]
 [  3  -4]
 [  0   0]
 [  2  -5]
 [ 28 -16]
 [  6  -4]
 [ 18 -15]
 [  2  -9]
 [  5   0]
 [  1   0]
 [ 10   0]
 [  0 -22]
 [  8   0]
 [  4  -3]
 [  5  -7]
 [ 10 -55]
 [  3  -8]
 [  5   0]
 [  7  -5]
 [ 20  -4]
 [  8 -28]
 [  6 -16]
 [  8 -31]
 [  4  -3]
 [ 12  -1]
 [  9  -9]
 [  4  -7]
 [  0   0]
 [  3  -9]
 [  5 -19]
 [  9   0]
 [ 39  -5]
 [  3   0]
 [  0  -3]
 [  3 -10]
 [  0  -2]
 [  0   0]
 [  3   0]
 [ 18 -25]
 [  4  -7]
 [  0   0]
 [  2  -2]
 [  3   0]
 [ 19  -4]
 [  0  -9]
 [  7  -6]
 [  0  -4]
 [ 11 -10]
 [  6   0]
 [  3  -8]
 [  7  -6]
 [  2 -10]
 [ 10  -4]
 [  3   0]
 [ 40 -22]
 [  9  -7]
 [  2   0]
 [  6   0]
 [  6   0]
 [  2 -13]
 [  3   0]
 [  0  -2]
 [  5   0]
 [  2 -14]
 [  5   0]
 [  2   0]
 [ 11  -8]
 [  5 -10]
 [  7   0]
 [  7  -2]
 [  3  -4]
 [  4   0]
 [  7  -2]
 [  7  -6]
 [ 15  -7]
 [ 12  -7]
 [  0  -3]
 [  7  -3]
 [  0 -20]
 [  5   0]
 [  5 -12]
 [  2 -23]
 [  6   0]
 [  7  -9]
 [  5   0]

In [126]:
# set tie to be positive
for i in range(senti_value.T.shape[0]):
    if senti_value.T[i,0] == - senti_value.T[i,1]:
        senti_output[i] = 1

In [127]:
comp = senti_output != data['sentiment'][0,:]
print("There are ", sum(comp)," conflicts.")
print("Indexes are:")
print(data['id'][0,np.where(comp)]+1)
print("Accuracy:",1 - sum(comp)/number_data)
for id in np.where(comp)[0]:
    print(data['id'][0,id]+1)
    print(senti_value.T[id,:])

There are  108  conflicts.
Indexes are:
[[  6   9  10  21  22  24  25  33  35  44  46  48  49  50  51  53  54  55
   57  58  60  61  63  64  69  73  77  79  80  81  91  94  95  99 103 107
  109 116 119 122 123 124 125 129 130 131 136 138 139 147 151 158 164 165
  166 168 175 177 178 179 180 184 188 190 193 194 196 199 205 207 210 211
  219 221 224 232 234 235 239 255 257 258 263 264 265 275 279 280 284 285
  286 287 290 291 292 296 297 300 302 305 312 314 317 321 332 334 335 336]]
Accuracy: 0.680473372781
6
[ 1 -2]
9
[ 1 -1]
10
[0 0]
21
[ 4 -3]
22
[ 2 -2]
24
[ 1 -2]
25
[0 0]
33
[ 2 -2]
35
[0 0]
44
[0 0]
46
[ 2 -2]
48
[0 0]
49
[ 1 -1]
50
[0 0]
51
[ 3 -1]
53
[ 1 -1]
54
[ 1 -2]
55
[ 3 -3]
57
[ 1 -1]
58
[ 1 -1]
60
[ 3 -1]
61
[0 0]
63
[ 2 -1]
64
[0 0]
69
[ 1 -2]
73
[0 0]
77
[ 2 -1]
79
[0 0]
80
[ 2 -1]
81
[ 2 -2]
91
[ 1 -2]
94
[ 2 -1]
95
[ 1 -1]
99
[ 2 -1]
103
[ 1 -3]
107
[ 2 -2]
109
[ 1 -1]
116
[ 2 -2]
119
[0 0]
122
[ 2 -1]
123
[ 2 -1]
124
[ 1 -2]
125
[ 1 -3]
129
[ 1 -3]
130
[ 1 -1]
131
[ 1

In [128]:
comp = senti_output != data['sentiment'][0,:]
print("There are ", sum(comp)," conflicts:")
print("Indexes are:")
print(data['id'][0,np.where(comp)]+1)
print("Accuracy:",1 - sum(comp)/number_data)

There are  108  conflicts:
Indexes are:
[[  6   9  10  21  22  24  25  33  35  44  46  48  49  50  51  53  54  55
   57  58  60  61  63  64  69  73  77  79  80  81  91  94  95  99 103 107
  109 116 119 122 123 124 125 129 130 131 136 138 139 147 151 158 164 165
  166 168 175 177 178 179 180 184 188 190 193 194 196 199 205 207 210 211
  219 221 224 232 234 235 239 255 257 258 263 264 265 275 279 280 284 285
  286 287 290 291 292 296 297 300 302 305 312 314 317 321 332 334 335 336]]
Accuracy: 0.680473372781


In [129]:
comp = senti_output != data['sentiment'][1,:]
print("There are ", sum(comp)," conflicts:")
print("Indexes are:")
print(data['id'][0,np.where(comp)]+1)
print("Accuracy:",1 - sum(comp)/number_data)

There are  90  conflicts:
Indexes are:
[[  6  15  21  22  24  33  34  36  44  47  51  53  54  55  57  58  60  61
   67  69  71  84  91  96  99 103 116 118 120 122 124 125 129 131 134 136
  138 139 142 143 151 153 155 167 175 176 177 182 188 194 196 198 199 200
  205 209 213 218 221 227 228 232 235 243 246 248 251 257 258 265 270 275
  278 280 284 286 287 292 296 297 299 302 305 310 315 322 324 332 335 336]]
Accuracy: 0.733727810651


              |  me     |   moritz
--------------|---------|-----------
paraAv+tiePos | 111(67%)|   97(71%)
paraAv+tieNeg | 137(59%)|   177(47%)
paraTot+tiePos| 99(70%) |   95(72%)
paraTot+tieNeg| 95(71%) |   115(66%)

1 NegatingMultiplier
paraAv+tiePos | 108(68%)|   90(73%)
paraAv+tieNeg | 143(57%)|   175(48%)
paraTot+tiePos| 99(70%) |   91(73%)
paraTot+tieNeg| 96(71%) |   114(66%)

In [102]:
senti_strength(" it might NOT be a bad thing.")

'0 0'

In [130]:
# initialize array for storing data
number_data = 338
data_1 = {"id": np.empty(number_data, dtype="int32"),
       "thread_id": np.empty(number_data, dtype="int32"),
       "sentiment": np.empty(number_data, dtype="int32"),
        "whoposts": np.empty(number_data, dtype="str"),
       "post_body": np.empty(number_data, dtype=object)}

In [131]:
# read annotations from .csv file
import csv
file = 'data/corrected_posts_delete.csv'
with open(file) as csvfile:
    reader = csv.reader(csvfile)
    count = 0
    for row in reader:
        # exclude the first row, which are column names
        if count != 0:
            #print("Current line:",count)
            data_1['id'][count-1] = count
            data_1['thread_id'][count-1] = row[0]
            data_1['sentiment'][count-1] = row[1]
            data_1['whoposts'][count-1] = row[2]
            data_1['post_body'][count-1] = row[3]
        count += 1

In [142]:
senti_value = np.empty([2,number_data],dtype="int32")
senti_output = np.empty(number_data,dtype="int32")
print(senti_value.shape)
for i in range(data_1['post_body'].shape[0]):
    output = senti_strength(data_1['post_body'][i])
    values = [int(x) for x in output.split(" ")]
    senti_value[:,i] = values
    if values[0]> -values[1]:
        senti_output[i] = 1
    else:
        senti_output[i] = 0
    if i % 30 == 0:
        print("predicted %d/%d" % (i, number_data))

(2, 338)
predicted 0/338
predicted 30/338
predicted 60/338
predicted 90/338
predicted 120/338
predicted 150/338
predicted 180/338
predicted 210/338
predicted 240/338
predicted 270/338
predicted 300/338
predicted 330/338


In [145]:
comp = senti_output != data_1['sentiment']
print("There are ", sum(comp)," conflicts:")
print("Indexes are:")
print(data['id'][0,np.where(comp)]+1)
print("Accuracy:",1 - sum(comp)/number_data)

There are  85  conflicts:
Indexes are:
[[  6   9  10  21  24  25  33  34  35  44  47  49  51  53  54  55  58  60
   61  69  73  78  81  84  91  95  99 103 109 116 118 122 123 124 125 129
  131 136 138 139 140 142 143 147 151 165 166 175 177 180 182 188 190 193
  194 196 205 207 208 209 218 221 232 234 235 236 251 255 257 258 263 265
  270 275 284 286 291 296 297 299 302 305 310 321 335]]
Accuracy: 0.748520710059


In [144]:
# set tie to be positive
for i in range(senti_value.T.shape[0]):
    if senti_value.T[i,0] == - senti_value.T[i,1]:
        senti_output[i] = 1

In [ ]:
              |  me     
--------------|---------
paraAv+tieNeg | 159(52%)
paraAn+tiePos | 88(74)
paraTot+tieNeg| 100(70%)
paraTot+tiePos| 85(75%)

In [147]:
data_1['sentiment']

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1,

In [148]:
np.sum(data_1['sentiment']==1)

210

In [149]:
np.sum(data_1['sentiment']==0)

128